## Sentiment Based Review Responder with Flan-T5

In this Notebook we want to use the output from our Sentiment Analysis model that we trained with the Yelp Dataset, to give a sentiment-based response to the reviews.

## Load the model

For this notebook, I am only going to use the LSTM model.

In [1]:
from tensorflow.keras.models import load_model

# Load the model
model_lstm = load_model('models/sentiment_analysis_model_lstm.h5')


2023-06-11 21:11:40.223599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 21:11:44.649338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-11 21:11:44.651424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split

Before we can use the model, we have to initate the tokenizer and preprocess the reviews:

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# If the preprocessing has already been done and you're ram is big enough, you can load the preprocessed data all at once
# But if your ram is not big enough, you can load the data in chunks and preprocess them in chunks under "Loading Preprocessed Data in Chunks"
df_preprocessed= pd.read_json('preprocessing/preprocessed_reviews_reduced_5k.json')

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 10000
# Max number of words in each review.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

# Extract the strings from the dictionaries in the 'text' column
df_preprocessed['text'] = df_preprocessed['text'].apply(lambda x: list(x.values())[0] if isinstance(x, dict) else x)

# Extract the ratings from the dictionaries in the 'stars' column
df_preprocessed['stars'] = df_preprocessed['stars'].apply(lambda x: list(x.values())[0] if isinstance(x, dict) else x)

# Create a tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@\[\]^_`{|}~', lower=True)

# Fit the tokenizer on the texts
tokenizer.fit_on_texts(df_preprocessed['text'].values)

# Vocabulary size
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# Transform text to sequence of integers
X = tokenizer.texts_to_sequences(df_preprocessed['text'].values)

Found 14673 unique tokens.


In [3]:
# Download the stopwords from NLTK
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

def preprocess_reviews(reviews):
    processed_reviews = []
    for review in reviews:
        # Convert to lowercase
        review = review.lower()
        # Remove punctuation
        review = re.sub(r'[^\w\s]', '', review)
        # Remove stopwords and stem the words
        review = ' '.join(stemmer.stem(word) for word in review.split() if word not in stop_words)
        processed_reviews.append(review)
    return processed_reviews

# Select a few reviews to test the model
test_reviews = [
    'The food was absolutely wonderful, from preparation to presentation, very pleasing.',
    'The staff did not give us good service.',
    'The restaurant was not clean. Our food was terrible.',
    'The food was delicious and the service was great!',
    'The food was ok. But we liked the service.',
    'We ate not fine, but the food was not great at all!'
]

# Preprocess the test reviews
test_reviews = preprocess_reviews(test_reviews)

# Convert the test reviews into sequences
test_sequences = tokenizer.texts_to_sequences(test_reviews)
test_sequences = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)


# Make predictions on the test reviews
predictions_lstm = model_lstm.predict(test_sequences)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dinopelesevic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-06-11 21:11:55.016036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-11 21:11:55.017273: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-11 21:11:55.018914: I tensorflow/core/common_runtime/executor.cc:1

1/1 [==============================] - 1s 538ms/step
[[0.20301124 0.19715099 0.20036684 0.2017606  0.19771037]
 [0.20172176 0.19951713 0.20218927 0.2005564  0.19601545]
 [0.20104069 0.1990636  0.19819582 0.20299238 0.19870758]
 [0.20035619 0.20165557 0.19830921 0.20424369 0.19543532]
 [0.20190658 0.20102546 0.20063524 0.19954455 0.19688813]
 [0.20139937 0.19881059 0.19969276 0.20307936 0.19701792]]


## Review Responder with Flan-T5

Based on the sentiment of the review, we will generate a response using the Flan-T5 model. If the review is positive, we will generate a positive response. If the review is negative, we will generate a negative response.

In [7]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import time
import numpy as np

# Load the model and tokenizer
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

# model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl") # If you have enough ram, you can use the XL model
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl") 

# Define a function that takes an input text and a star rating and generates a response
def generate_response(input_text, stars):
    start_time = time.time()

    # Add conditioning text based on the number of stars
    if stars == 1:
        prefix = "You're a restaurant owner and got the following review. Give answer and apologize:"
    elif stars == 2:
        prefix = "You're a restaurant owner and got the following review. Give answer and apologize:"
    elif stars == 3:
        prefix = "You're a restaurant owner and got the following review:"
    elif stars == 4:
        prefix = "You're a restaurant owner and got the following review. Give a thankful answer:"
    elif stars == 5:
        prefix = "You're a restaurant owner and got the following review. You're very happy and want to thank the customer:"

    input_text = f"{prefix} {input_text}"

    inputs = tokenizer(input_text, return_tensors="tf")
    outputs = model.generate(
        **inputs,
        min_length=0,
        max_length=512,  # Adjust as needed
        length_penalty=2.0,
        num_beams=16,
        no_repeat_ngram_size=2,
        early_stopping=True,
    )

    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    end_time = time.time()
    total_time = end_time - start_time

    print(f"Output: {output_text[0]}, Rating: {stars}")
    print(f"Inference time: {total_time} seconds")


/Users/dinopelesevic/Documents/ML2/ML2-Project/venv/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/flan-t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Testing

Now we can test the sentiment-based review-response:

In [8]:
# Select a few reviews to test the model
reviews = [
    'The food was absolutely wonderful, from preparation to presentation, very pleasing.',
    'The staff did not give us good service.',
    'The restaurant was not clean. Our food was terrible.',
    'The food was delicious and the service was great!',
    'The food was ok. But we liked the service.',
    'We ate not fine, but the food was not great at all!'
]

# Print the predictions
for i, review in enumerate(reviews):
    generate_response(review, predictions_lstm[i])


Output: This is a great place to eat!, Rating: 3
Inference time: 3.8322629928588867 seconds
Output: The staff did not give us good service, Rating: 1
Inference time: 3.2041401863098145 seconds
Output: The food was mediocre., Rating: 5
Inference time: 3.346299171447754 seconds
Output: This is a great restaurant!, Rating: 4
Inference time: 3.4511916637420654 seconds
Output: The food was okay. But we liked the service., Rating: 2
Inference time: 4.715893983840942 seconds
Output: The food was not great at all, Rating: 5
Inference time: 3.043585777282715 seconds


# Interpretation

The answers are generated and the code works, but the quality of the answers is very insufficient. This is due to the Flan-T5-Small model, which is very small.

The Flan-T5-XXL model is much better trained and has a better quality in the generation of text. Attached are two screenshots of the Huggingface hosted API:

![1-StarReview](../images/1-StarReview.png)

![5-Star Review](../images/5-StarReview.png)